In [3]:
import pulp
import pandas as pd

In [7]:
df_ava = pd.read_csv("shift_ava.csv", sep="\t")
df_ava

,Time,A,B,C,D
0,Morning,1,1,0,1
1,Afternoon,1,0,1,1
2,Evening,0,1,1,1


In [17]:
df_req = pd.read_csv("shift_req.csv")
df_req

,Time,req
0,Morning,2
1,Afternoon,2
2,Evening,1


In [23]:
df_ava2 = df_ava.copy()
df_ava2.index = df_ava["Time"]
df_ava2 = df_ava2.drop("Time", axis=1)
df_ava2

,A,B,C,D
Time,,,,
Morning,1,1,0,1
Afternoon,1,0,1,1
Evening,0,1,1,1


In [27]:
df_off = pd.read_csv("shift_off.csv")
df_off

,name,day
0,A,3
1,B,4
2,C,5
3,D,6
4,A,8
5,B,15
6,D,22


In [113]:
holiday = [7, 14, 21, 28]
days = list(range(1, 32))

In [31]:
incompatible_pair = [("A", "B")]

In [233]:
emp = df_ava2.columns
shift = df_ava["Time"].values
ava = {}
for col in df_ava2.columns:
    ind = df_ava2.index
    ava[col] = {ind[0]: df_ava2.loc[ind[0], col], ind[1] : df_ava2.loc[ind[1], col], ind[2] : df_ava2.loc[ind[2], col]}
req = dict(df_req.values)
off = {}
for i in range(len(df_off.values)):
    try:
        off[df_off.values[i][0]].append(df_off.values[i][1])
    except:
        off[df_off.values[i][0]] = [df_off.values[i][1]]

In [439]:
print(emp)
print(shift)
print(ava)
print(req)
print(off)

Index(['A', 'B', 'C', 'D'], dtype='object')
['Morning' 'Afternoon' 'Evening']
{'A': {'Morning': 1, 'Afternoon': 1, 'Evening': 0}, 'B': {'Morning': 1, 'Afternoon': 0, 'Evening': 1}, 'C': {'Morning': 0, 'Afternoon': 1, 'Evening': 1}, 'D': {'Morning': 1, 'Afternoon': 1, 'Evening': 1}}
{'Morning': 2, 'Afternoon': 2, 'Evening': 1}
{'A': [3, 8], 'B': [4, 15], 'C': [5], 'D': [6, 22]}


In [634]:
prob = pulp.LpProblem("shift", pulp.LpMinimize)

In [636]:
x = pulp.LpVariable.dicts("shift_num", [(e, s, d) for e in emp for s in shift for d in days], cat="Binary")
prob += pulp.lpSum((1-ava[e][s]*x[(e, s, d)] for e in emp for s in shift for d in days))

In [638]:
for d in days:
    if d in holiday:
        for s in shift:
            prob += pulp.lpSum(x[e, s, d] for e in emp) == 0
    else:
        for s in shift:
            prob += pulp.lpSum(x[e, s, d] for e in emp) >= req[s]

In [640]:
for e in emp:
    for d in days:
        prob += pulp.lpSum(x[e, s, d] for s in shift) <= 2

In [642]:
for e, days_off in off.items():
    for d in days_off:
        _=1
        prob += pulp.lpSum(x[e, s, d] for s in shift) == 0

In [644]:
for d in days:
    for s in shift:
        for pair in incompatible_pair:
            e1, e2 = pair
            prob += x[(e1, s, d)] + x[(e2, s, d)] <= 1

In [648]:
prob.solve()

1

In [650]:
print("Status:", pulp.LpStatus[prob.status])

Status: Optimal


In [656]:
print("Assign:")
for d in days:
    print("Day %d"%(d))
    for s in shift:
        assign = []
        for e in emp:
            if x[(e, s, d)].value() == 1:
                assign.append(e)
        print(s, assign) if assign else "No one assign"

Assign:
Day 1
Morning ['B', 'D']
Afternoon ['A', 'C', 'D']
Evening ['B', 'C']
Day 2
Morning ['B', 'D']
Afternoon ['A', 'C', 'D']
Evening ['B', 'C']
Day 3
Morning ['B', 'D']
Afternoon ['C', 'D']
Evening ['B', 'C']
Day 4
Morning ['A', 'D']
Afternoon ['A', 'C']
Evening ['C', 'D']
Day 5
Morning ['B', 'D']
Afternoon ['A', 'D']
Evening ['B']
Day 6
Morning ['A', 'C']
Afternoon ['A', 'C']
Evening ['B']
Day 7
Day 8
Morning ['B', 'D']
Afternoon ['C', 'D']
Evening ['B', 'C']
Day 9
Morning ['A', 'D']
Afternoon ['A', 'C']
Evening ['B', 'C', 'D']
Day 10
Morning ['A', 'D']
Afternoon ['A', 'C', 'D']
Evening ['B', 'C']
Day 11
Morning ['B', 'D']
Afternoon ['A', 'C', 'D']
Evening ['B', 'C']
Day 12
Morning ['A', 'D']
Afternoon ['A', 'C']
Evening ['B', 'C', 'D']
Day 13
Morning ['A', 'D']
Afternoon ['A', 'C', 'D']
Evening ['B', 'C']
Day 14
Day 15
Morning ['A', 'D']
Afternoon ['A', 'C', 'D']
Evening ['C']
Day 16
Morning ['A', 'D']
Afternoon ['A', 'C', 'D']
Evening ['B', 'C']
Day 17
Morning ['B', 'D']
Afterno

In [658]:
import pulp

# データの定義
employees = ["Alice", "Bob", "Charlie", "Diana"]
shifts = ["Morning", "Afternoon", "Evening"]
days = list(range(1, 32))  # 1日〜31日

# 各シフトに必要な最低人数
shift_requirements = {
    "Morning": 2,
    "Afternoon": 2,
    "Evening": 1
}

# 従業員ごとのシフト希望（1: 希望する, 0: 希望しない）
availability = {
    "Alice": {"Morning": 1, "Afternoon": 1, "Evening": 0},
    "Bob": {"Morning": 1, "Afternoon": 0, "Evening": 1},
    "Charlie": {"Morning": 0, "Afternoon": 1, "Evening": 1},
    "Diana": {"Morning": 1, "Afternoon": 1, "Evening": 1}
}

# 休日設定（指定の日は全員休み）
holidays = [7, 14, 21, 28]

# 希望休設定（特定の日に休みたい従業員）
day_off_requests = {
    "Alice": [3, 8],
    "Bob": [4, 15],
    "Charlie": [5],
    "Diana": [6, 22]
}

# 誰と誰を組み合わせてはいけない（例: AliceとBobは同じシフトで働けない）
incompatible_pairs = [("Alice", "Bob")]

# 問題の定義
prob = pulp.LpProblem("Monthly_Shift_Scheduling_With_Constraints", pulp.LpMinimize)

# 変数の定義
x = pulp.LpVariable.dicts("Shift", [(e, s, d) for e in employees for s in shifts for d in days],
                          cat="Binary")

# 目的関数: 希望に沿わないシフトの割り当てを最小化
prob += pulp.lpSum((1 - availability[e][s]) * x[(e, s, d)]
                   for e in employees for s in shifts for d in days)

# 制約1: 各シフトに必要な最低人数を確保
for d in days:
    if d in holidays:
        for s in shifts:
            prob += pulp.lpSum(x[(e, s, d)] for e in employees) == 0  # 休日は全員休み
    else:
        for s in shifts:
            prob += pulp.lpSum(x[(e, s, d)] for e in employees) >= shift_requirements[s]

# 制約2: 各従業員は1日に1つのシフトにしか入れない
for e in employees:
    for d in days:
        prob += pulp.lpSum(x[(e, s, d)] for s in shifts) <= 1

# 制約3: 希望休を考慮（特定の日には割り当てられない）
for e, days_off in day_off_requests.items():
    for d in days_off:
        prob += pulp.lpSum(x[(e, s, d)] for s in shifts) == 0

# 制約4: 特定のペアが同じシフトに割り当てられない
for d in days:
    for s in shifts:
        for pair in incompatible_pairs:
            e1, e2 = pair
            prob += x[(e1, s, d)] + x[(e2, s, d)] <= 1

# 制約5: 連続勤務の制限（連続3日以上の勤務を禁止）
for e in employees:
    for d in range(1, 30):  # 31日-2日の範囲
        prob += (x[(e, "Morning", d)] + x[(e, "Afternoon", d)] + x[(e, "Evening", d)]
                 + x[(e, "Morning", d+1)] + x[(e, "Afternoon", d+1)] + x[(e, "Evening", d+1)]
                 + x[(e, "Morning", d+2)] + x[(e, "Afternoon", d+2)] + x[(e, "Evening", d+2)]) <= 2

# 解を求める
prob.solve()

# 結果の出力
print("Status:", pulp.LpStatus[prob.status])
print("\nShift Assignments:")
for d in days:
    print(f"\nDay {d}:")
    for s in shifts:
        assigned = []
        for e in employees:
            if x[(e, s, d)].value() == 1:
                assigned.append(e)
        print(f"  {s}: {', '.join(assigned) if assigned else 'No one assigned'}")


Status: Infeasible

Shift Assignments:

Day 1:
  Morning: Bob, Diana
  Afternoon: No one assigned
  Evening: No one assigned

Day 2:
  Morning: No one assigned
  Afternoon: Alice, Charlie
  Evening: No one assigned

Day 3:
  Morning: No one assigned
  Afternoon: Charlie, Diana
  Evening: No one assigned

Day 4:
  Morning: No one assigned
  Afternoon: Alice, Diana
  Evening: No one assigned

Day 5:
  Morning: Bob, Diana
  Afternoon: Diana
  Evening: No one assigned

Day 6:
  Morning: No one assigned
  Afternoon: Alice, Charlie
  Evening: No one assigned

Day 7:
  Morning: No one assigned
  Afternoon: No one assigned
  Evening: No one assigned

Day 8:
  Morning: Bob, Diana
  Afternoon: No one assigned
  Evening: No one assigned

Day 9:
  Morning: No one assigned
  Afternoon: No one assigned
  Evening: No one assigned

Day 10:
  Morning: Bob
  Afternoon: Alice
  Evening: No one assigned

Day 11:
  Morning: Bob, Diana
  Afternoon: Bob, Diana
  Evening: No one assigned

Day 12:
  Morning: A